In [1]:
import os
import numpy as np
import lsst
from lsst.daf.butler import Butler
from lsst.ip.diffim import GetTemplateConfig, GetTemplateTask

In [2]:
!pwd

/pscratch/sd/s/shl159/projects/update_plots/dia_improvement/devel


In [3]:
def htm_from_ra_dec_level(ra, dec, level):
    pixelization = lsst.sphgeom.HtmPixelization(level)
    htm_id = pixelization.index(
        lsst.sphgeom.UnitVector3d(
            lsst.sphgeom.LonLat.fromDegrees(ra, dec)
        )
    )
    return htm_id

def get_dataset_refs_from_htm_list(dataset_type, htm_ids, level, band, aggregate="intersection"):
    hi = htm_ids[0]

    # dataset_refs is an iterator, but each query is only a few hundred results,
    #   so convert to a list for future convenience
    htm_kwargs = {}
    htm_kwargs[f"htm{level}"] = hi
    dataset_refs = list(butler.registry.queryDatasets(dataset_type, dataId={"band": band}, **htm_kwargs))
    dataset_refs = set(dataset_refs)
    
    for hi in htm_ids[1:]:
        htm_kwargs = {}
        htm_kwargs[f"htm{level}"] = hi
        dr = list(butler.registry.queryDatasets(dataset_type, dataId={"band": band}, **htm_kwargs))
        if aggregate == "intersection":
            dataset_refs = dataset_refs.intersection(set(dr))
        elif aggregate == "union":
            dataset_refs = dataset_refs.union(set(dr))
        else:
            print("Aggregation method '{aggregate}' not supported.")
            return
        
    return list(dataset_refs)

In [4]:
def get_buter(output_collection='user', site='NERSC'):
    # survey_site = {"USDF": "HSC", "IDF": "DC2", "NERSC": "DC2"}
    # ra_dec_survey = {"HSC": (150, 2.5), "DC2": (55, -30)}
    if output_collection == 'user':
        user = os.getenv("USER")
        output_collection = f"u/{user}/dia_science"        
    
    collection_site = {"USDF": "HSC/runs/RC2/w_2023_15/DM-38691", "IDF": "2.2i/runs/DP0.2", "NERSC": "u/descdm/coadds_Y1_4638"}
    collection = collection_site[site]
    repo_site = {"USDF": "/repo/main", "IDF": "dp02", "NERSC": "/global/cfs/cdirs/lsst/production/gen3/DC2/Run2.2i/repo"}
    repo_config = repo_site[site]
    butler = Butler(repo_config, run=output_collection, collections=[output_collection, collection])
    return butler

def get_science_ref(ra, dec, band):
    level = 20  # the resolution of the HTM grid
    htm_id = htm_from_ra_dec_level(ra, dec, level)
    parent_level = htm_id // 10
    htm_ids = [parent_level * 10 + i for i in [0, 1, 2, 3]]
    dataset_refs = get_dataset_refs_from_htm_list("calexp", htm_ids, level, band='i')
    # Sort by visitId to get a loose time order
    ids_visit = [dr.dataId["visit"] for dr in dataset_refs]
    dataset_refs = [dataset_refs[idx] for idx in np.argsort(ids_visit)]
    
    return dataset_refs

def get_template_ref(ra, dec, band):
    bigger_level = 9
    bigger_htm_id = htm_from_ra_dec_level(ra, dec, level=bigger_level)
    coadd_exposure_refs = get_dataset_refs_from_htm_list("deepCoadd", [bigger_htm_id], level=bigger_level, band=band, aggregate="union")
    return coadd_exposure_refs

def get_template(template_refs, science, butler):
    get_template_task_config = GetTemplateConfig()
    get_template_task = GetTemplateTask(config=get_template_task_config)
    
    coadd_exposure_deferred_dataset_handles = [butler.getDeferred(dr) for dr in template_refs]
    sky_map = butler.get("skyMap")
    inputs = {"coaddExposures" : coadd_exposure_deferred_dataset_handles,
              "bbox": science.getBBox(),
              "skyMap": sky_map,
              "wcs": science.getWcs(),
              "visitInfo": science.visitInfo,
             }

    results = get_template_task.getOverlappingExposures(inputs)
    coadd_exposures = results.coaddExposures
    data_ids = results.dataIds

    deep_coadd_template = get_template_task.run(coadd_exposures, inputs["bbox"], inputs["wcs"], data_ids)
    template = deep_coadd_template.template
    return template

In [5]:
4638

4638

In [6]:
4000 * 0.2 / 3600

0.2222222222222222

In [7]:
(55.2 - 53.9) / 0.222

5.855855855855875

In [8]:
butler = get_buter(output_collection='user', site='NERSC')

ra = [53.6,55.5], [53.9, 55.2]

dec [-31.4, -29.6], [-31.2, -29.9]

1. test dia with piecewise exposures
2. 100 image pairs. 
3. select 

In [9]:
4000 * 0.2 / 3600

0.2222222222222222

patch 00 -> patch 66



(53.6, -31.2) -> (55.2, -29.6)

In [10]:
(55.5 - 53.7) / 0.22, (-29.7 + 31.2) / 0.22

(8.18181818181817, 6.818181818181818)

ra_range = np.linspace(53.9, 55.2, 10).round(2)
dec_range = np.linspace(-31.2, -29.9, 10).round(2)

In [11]:
ra_range = np.linspace(53.7, 55.5, 7).round(2)
dec_range = np.linspace(-31.2, -29.7, 7).round(2)

injection strategy


- broad vs. sharp

    - mag 20, host 20

    - mag 23, host 20

    - mag 20, host 23

    - mag 23, host 23
    
- how many hosts?

- how many pieces, 10 * 10? 8 * 8

In [14]:
4000 * 0.2 / 3600

0.2222222222222222

In [13]:
(55.5 - 53.7) / 0.22, (-29.7 + 31.2) / 0.22

(8.18181818181817, 6.818181818181818)

1. injections per image (m)

2. how many science images (n)

3. m * n

In [ ]:
ra_range = np.linspace(53.7, 55.5, 7).round(2)
dec_range = np.linspace(-31.2, -29.7, 7).round(2)

In [171]:
calexp_ids = set()
band = 'i'
result = []
for ra in ra_range:
    for dec in dec_range:
        science_refs = get_science_ref(ra, dec, band)
        template_refs = get_template_ref(ra, dec, band)
        i = 0
        while science_refs[i].dataId in calexp_ids:
            i += 1
        science = butler.get("calexp", science_refs[i].dataId)
        calexp_ids.add(science_refs[i].dataId)
        x = science.getWcs().getSkyOrigin().getRa().asDegrees()
        y = science.getWcs().getSkyOrigin().getDec().asDegrees()
        print(len(science_refs), len(template_refs), x,y)
        result.append([len(science_refs), len(template_refs)])
    print('\n')

15 1 53.570555301373844 -31.168315280377694
10 2 53.70179955700191 -30.99584391697023
14 2 53.7268850446794 -30.674108889379628
13 3 53.68777890033913 -30.329658401617404
19 2 53.57938759071659 -30.169415269758208
21 2 53.659056531199596 -29.84290263184492
24 1 53.72714338727883 -29.635023567682374


16 2 53.971668775190004 -31.07727818912097
16 4 54.021516843487866 -30.926912029980098
13 6 53.90462125053126 -30.594625160665082
15 4 53.88900311614404 -30.478825875172664
16 4 53.9356121941926 -30.08548154988078
19 4 53.973525459123664 -29.918459453968968
18 1 54.070257773535744 -29.69958944852883


18 4 54.31452331804923 -31.10274120166769
19 6 54.3465673991383 -30.860479840422276
18 3 54.353951957631935 -30.598671190651288
18 5 54.3996250261985 -30.370217258579153
18 4 54.44073480655596 -30.120080933288094
16 4 54.222263472947255 -30.012276725018282
18 2 54.33202393304478 -29.78407145069952


19 4 54.59091711496539 -31.141410413085058
18 3 54.62981053504519 -30.888395412904767
18 3 54.

In [ ]:
(55.2 - 53.6) / 0.22

In [93]:
ra, dec = 55.2, -29.6
band = 'i'

science_refs = get_science_ref(ra, dec, band)
template_refs = get_template_ref(ra, dec, band)

In [94]:
print(len(science_refs), len(template_refs))

26 1


In [95]:
template_refs[0].dataId['patch']

42

In [13]:
(55.2 - 53.9) / 10

0.13000000000000042

In [14]:
np.linspace(53.9, 55.2, 10).round(2)

array([53.9 , 54.04, 54.19, 54.33, 54.48, 54.62, 54.77, 54.91, 55.06,
       55.2 ])

In [12]:
ra_range = np.linspace(53.9, 55.2, 10).round(2)
dec_range = np.linspace(-31.2, -29.9, 10).round(2)

In [14]:
ra_range, dec_range

(array([53.9 , 54.04, 54.19, 54.33, 54.48, 54.62, 54.77, 54.91, 55.06,
        55.2 ]),
 array([-31.2 , -31.06, -30.91, -30.77, -30.62, -30.48, -30.33, -30.19,
        -30.04, -29.9 ]))

In [15]:
band = 'i'
result = []
for ra in ra_range:
    for dec in dec_range:
        science_refs = get_science_ref(ra, dec, band)
        template_refs = get_template_ref(ra, dec, band)
        print(len(science_refs), len(template_refs))
        result.append([len(science_refs), len(template_refs)])
    print('\n')

13 2
12 4
14 4
12 5
14 5
14 4
14 4
16 4
16 3
17 4


14 2
18 4
16 7
15 6
13 6
16 4
14 4
18 4
19 3
19 4


19 2
16 4
17 6
14 4
15 4
14 3
16 4
14 4
19 3
21 4


20 3
17 4
17 4
18 3
17 3
20 5
16 4
18 4
16 4
18 4


18 3
19 4
22 3
19 3
18 4
16 4
18 4
19 4
19 4
20 4


18 4
19 4
20 4
18 3
21 3
16 5
17 5
18 4
19 4
21 4


18 4
19 6
18 4
20 3
26 4
20 3
18 3
19 4
21 4
19 4


17 6
17 6
20 4
24 4
24 4
20 4
18 4
18 3
18 4
24 4


19 4
20 4
23 4
22 4
23 4
24 4
22 4
19 4
23 4
20 4


20 4
23 4
21 3
22 4
24 4
20 4
18 4
20 4
21 4
22 4




In [308]:
(-29.9 + 31.2) / delta_degree

2.6000000000000014

In [309]:
delta_degree

0.5

In [310]:
4000 * 0.4 / 3600

0.4444444444444444

In [184]:
science = butler.get("calexp", science_refs[0].dataId)
template = get_template(template_refs, science, butler)

In [311]:
science.getImage().array.shape

(4000, 4072)

In [110]:
science.getWcs()

FITS standard SkyWcs:
Sky Origin: (57.0735012847, -29.4345321171)
Pixel Origin: (1864.75, 2018.68)
Pixel Scale: 0.200029 arcsec/pixel

In [108]:
science.getImage().array.shape

(4000, 4072)

In [67]:
4100 *0.4 / 3600

0.45555555555555555

In [69]:
delta_degree = 0.46

In [45]:
science_refs = get_science_ref(ra=55+delta_deg, dec=-30, output_collection='user', band = "i", site='NERSC')

In [90]:
science_refs[0]

DatasetRef(DatasetType('calexp', {band, instrument, detector, physical_filter, visit_system, visit}, ExposureF), {instrument: 'LSSTCam-imSim', detector: 83, visit: 6826, band: 'i', physical_filter: 'i_sim_1.4', visit_system: 1}, run='u/descdm/sfp_Y1_4431_4432_visits_part_00/20211217T211930Z', id=d4d73fb2-b783-47c7-9569-ceab92502f52)

In [86]:
ra_center, dec_center = 55, -30
delta_degree = 0.5
result = []
for i in range(-5, 5):
    delta_ra = delta_degree * i
    for j in range(-8, 2):
        delta_dec = delta_degree * j
        
        ra = ra_center + delta_ra
        dec = dec_center + delta_dec
        science_refs = get_science_ref(ra, dec)
        result.append(len(science_refs))
        science = butler.get("calexp", science_refs[0].dataId)
        print(len(science_refs), science.getWcs().getSkyOrigin())
    print('\n')

16 (52.5037075144, -33.9568219186)
16 (52.6271917541, -33.5178813765)
16 (52.4964947767, -32.9944196029)
24 (52.6500871914, -32.5265933615)
22 (52.5050075329, -32.0200451363)
20 (52.4254671358, -31.5841318903)
17 (52.5910575890, -31.0650900071)
16 (52.5381625185, -30.4015184072)
17 (52.5117205765, -30.0809372390)
19 (52.4590778271, -29.4158667168)


15 (53.0543878582, -34.0627356700)
20 (52.9139099594, -33.5460795478)
16 (53.0423019912, -33.0966075907)
22 (52.9087917036, -32.5869587976)
21 (53.0557943110, -32.1227314914)
18 (53.0101574849, -31.3749588041)
15 (52.9823882395, -31.0358387407)
12 (53.1415554210, -30.5307878185)
17 (52.8825511834, -30.0498140727)
17 (52.9273606424, -29.4656599854)


13 (53.6640640024, -33.9538140307)
16 (53.5306856950, -33.4329467321)
16 (53.3884726947, -32.9339752343)
20 (53.5259548934, -32.4765780099)
19 (53.3703206043, -31.9603936929)
17 (53.5195096468, -31.4974172759)
10 (53.3711790887, -31.0306678279)
13 (53.5134305762, -30.5033028091)
18 (53.464292813

In [ ]:
ra_range  = (-5, 5) # [-6, 9]
dec_range = (-8, 2) # [-9, 4]

In [18]:
# science = butler.get("calexp", science_dr.dataId)
# source_catalog = butler.get("src", dataId=science_dr.dataId)

In [21]:
len(dataset_refs)

22

In [22]:
bbox = science.getBBox()

In [31]:
# bbox.contains()
wcs = science.getWcs()
wcs.getSkyOrigin()

SpherePoint(55.04636806215645*degrees, -30.00032343802074*degrees)

In [73]:
for science_dr in science_refs:
    science = butler.get("calexp", science_dr.dataId)
    print(science.getWcs().getSkyOrigin())

(54.9324413557, -30.4338816571)
(54.9861575042, -30.4600072996)
(54.9667397103, -30.3538252888)
(55.0615899612, -30.5568152211)
(55.0413578239, -30.5122344235)
(54.9591418334, -30.5335349529)
(54.9427623097, -30.3911937606)
(55.0686517377, -30.3439233089)
(55.0790461113, -30.4922088877)
(54.9135411726, -30.3834550884)
(54.9354553243, -30.4111710646)
(55.0253824154, -30.4250243018)
(55.0199947648, -30.3137194808)
(54.8598071950, -30.5127807284)
(54.9577241699, -30.3965942767)
(54.9359836867, -30.4467367617)
(55.0110711860, -30.4470850546)
(54.8540541615, -30.4758640657)
(54.9975772864, -30.5710063253)
(55.0144907438, -30.3462592589)
(54.9009024774, -30.5577980115)
(54.9021331902, -30.4993412321)
